# Вводная часть

## Импорт библиотек

In [1]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import requests 
from urllib.parse import urlencode 
import re
from calendar import day_name

%matplotlib inline
# импорт библиотеки подключения к яндекс-метрике
from tapi_yandex_metrika import YandexMetrikaStats
from pandas.io.json import json_normalize
import json
from requests_html import HTMLSession
session = HTMLSession()
from pandas import Series,DataFrame
from io import BytesIO
from time import sleep
import sys

In [2]:
# Метод для корректной обработки строк в кодировке UTF-8 как в Python 3, так и в Python 2
if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode("utf8")
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if type(x) == type(b''):
            return x.decode('utf8')
        else:
            return x

## Функция загрузки данных

In [3]:
def req_to_direct(URL, body, headers):   
    # --- Запуск цикла для выполнения запросов ---
    # Если получен HTTP-код 200, то выводится содержание отчета
    # Если получен HTTP-код 201 или 202, выполняются повторные запросы
    while True:
        try:
            req = requests.post(URL, body, headers=headers)
            req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
            if req.status_code == 400:
                print("Параметры запроса указаны неверно или достигнут лимит отчетов в очереди")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(u(body)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 200:
                print("Отчет создан успешно")
                format(u(req.text))
                return(req.text)
                # return(req.content) если вдруг хотим в байтах выгрузить и преобразовывать
                break
            elif req.status_code == 201:
                print("Отчет успешно поставлен в очередь в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 202:
                print("Отчет формируется в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 500:
                print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 502:
                print("Время формирования отчета превысило серверное ограничение.")
                print("Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
                print("JSON-код запроса: {}".format(body))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            else:
                print("Произошла непредвиденная ошибка")
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(body))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break

        # Обработка ошибки, если не удалось соединиться с сервером API Директа
        except ConnectionError:
            # В данном случае мы рекомендуем повторить запрос позднее
            print("Произошла ошибка соединения с сервером API")
            # Принудительный выход из цикла
            break

        # Если возникла какая-либо другая ошибка
        except:
            # В данном случае мы рекомендуем проанализировать действия приложения
            print("Произошла непредвиденная ошибка")
            # Принудительный выход из цикла
            break

# Запрос к директу

## Токен доступа

In [4]:
# токен доступа к директу
ACCESS_TOKEN = "y0_AgAAAAADW_NUAAj-6gAAAADZkOljtoPASx4mRcSALt07Rr16GnaguH8"

ACCESS_TOKEN_EZ = "y0_AgAAAABn3RLQAAj-6gAAAADcLZHlv7hHebEIS8Wkthe5CfGgX8rNW04"

## Параметры запроса

In [5]:
# --- Входные данные ---
# Адрес сервиса Reports для отправки JSON-запросов (регистрозависимый)
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

#адрес сервиса ОТЧЕТОВ ПЕСОЧНИЦА
ReportsURL_SANDBOX = 'https://api-sandbox.direct.yandex.com/json/v5/reports'

# OAuth-токен пользователя, от имени которого будут выполняться запросы
token = ACCESS_TOKEN_EZ

# Логин клиента рекламного агентства
# Обязательный параметр, если запросы выполняются от имени рекламного агентства
# это тот логин, для которого получен токен!!!
clientLogin = 'evgeny.zhmuidetsky'

#какие столбцы подгрузить
# для дашборда необходимы дата, тип рекламной сети, показы, клики, стоимость, количество конверсий
stolbec = ["Date", "AdNetworkType", "Impressions", "Clicks", "Cost", "Conversions"]


#даты отчета
#StartDate = "2022-01-03"
#FinishDate = "2022-01-05"

# --- Подготовка запроса ---
# Создание HTTP-заголовков запроса
headers = {
           # OAuth-токен. Использование слова Bearer обязательно
           "Authorization": "Bearer " + token,
           # Логин клиента рекламного агентства
           "Client-Login": clientLogin,
           # Язык ответных сообщений
           "Accept-Language": "ru",
           # Режим формирования отчета
           "processingMode": "auto",
           # Формат денежных значений в отчете, если не сделать, то расходы будут в миллионах
            "returnMoneyInMicros": "false",
           # Не выводить в отчете строку с названием отчета и диапазоном дат
           # "skipReportHeader": "true",
           # Не выводить в отчете строку с названиями полей
            #"skipColumnHeader": "true",
           # Не выводить в отчете строку с количеством строк статистики
            "skipReportSummary": "true"
           }

# Создание тела запроса
body = {
    "params": {
        # критерии выбора, тут могут быть фильтры, логические операторы, 
        "SelectionCriteria": 
              {
              }, 
        "FieldNames": stolbec,
        "ReportName": u("Отчет 99"),
        "ReportType": "CUSTOM_REPORT",   # тип отчета
        "DateRangeType": "YESTERDAY",
        "Format": "TSV",  # обязательно
        "IncludeVAT": "YES",
        "IncludeDiscount": "NO"
              }
       }

# Кодирование тела запроса в JSON
body = json.dumps(body, indent=4)


## Запрос к директу

In [6]:
# запрос к данным директа
result = req_to_direct(ReportsURL, body, headers)

Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 7726378136019940464
Отчет создан успешно


## Преобразование ответа в датафрейм

In [7]:
# Преобразование ответа в датафрейм   ()      
file = open("cashe.csv", "w")   # сздаем и открываем новый файл
file.write(result)            # записываем в него ответ API
file.close()                    # закрываем файл
df = pd.read_csv("cashe.csv", header=1, sep='\t', index_col=0, encoding='cp1251')     # записываем в датафрейм файл
df = df.reset_index()
df.head()

,Date,AdNetworkType,Impressions,Clicks,Cost,Conversions
0,2023-02-13,AD_NETWORK,15107,67,29.83,--
1,2023-02-13,SEARCH,81,5,200.78,1


## Преобразование форматов

In [8]:
# Столбец даты переводить в формат даты нельзя, это мешает записи в файл
# столбец конверсий в формат целых чисел
df['Conversions'] = df['Conversions'].replace('--', '0')
df['Conversions'] = df['Conversions'].astype('int')

## Подсчет CTR,  средней стоимости клика и конверсии

In [9]:
df

,Date,AdNetworkType,Impressions,Clicks,Cost,Conversions
0,2023-02-13,AD_NETWORK,15107,67,29.83,0
1,2023-02-13,SEARCH,81,5,200.78,1


In [10]:
df['Aver_click_cost'] = round(df['Cost'] / df['Clicks'] , 2)
df['CTR'] = round((df['Clicks'] / df['Impressions']) *100 , 2)

# посчитаем стоимость целевого действия, если его  не было то оно = 0
df.loc[df['Conversions'] == 0, 'Aver_target_cost'] = 0
df.loc[df['Conversions'] != 0, 'Aver_target_cost'] = round(df['Cost'] / df['Conversions'] , 2)
#df['Aver_terget_cost'] = round(df['Cost'] / df['Conversions'] , 2)

# Загрузка файла на Гугл диск

## Загрузка библиотек

In [11]:
# Подключаем библиотеки
import httplib2 
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials

from __future__ import print_function
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
import pprint
import io

pp = pprint.PrettyPrinter(indent=4)

# библиотека работы с Гугл шитс
# https://docs.gspread.org/en/latest/user-guide.html#opening-a-spreadsheet
import gspread
import gspread_dataframe as gd

## Файл с ключом доступа

In [12]:
# Ссылка на файл с закрытым ключом
CREDENTIALS_FILE = r'C:\Users\ezhmui04\Documents\Ключ доступа к GoogleCloud\directdashboard-734cbb03b84d.json' 
# создание SCOPE только для драйва
SCOPES_drive = ['https://www.googleapis.com/auth/drive']

In [13]:
credentials_drive = service_account.Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=SCOPES_drive)
gc = gspread.service_account(CREDENTIALS_FILE)

## Запись датафрейма в файл

In [14]:
# Читаем ключи из файла
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])

httpAuth = credentials.authorize(httplib2.Http()) # Авторизуемся в системе
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth) # Выбираем работу с таблицами и 4 версию API

In [15]:
#df['Date'] = df['Date'].astype(str)

In [16]:
# Open a spreadsheet by title
sh = gc.open("Data_for_dash")
wk = sh.sheet1

params = {'valueInputOption': 'USER_ENTERED'}
body = {'values': df.values.tolist()}
sh.values_append(f'wkA1:D1', params, body)

{'spreadsheetId': '193gD3jEMjezNXZDLg93RsDfV-UTbshsyNWODVXPQJNE',
 'tableRange': 'List1!A1:I452',
 'updates': {'spreadsheetId': '193gD3jEMjezNXZDLg93RsDfV-UTbshsyNWODVXPQJNE',
  'updatedRange': 'List1!A453:I454',
  'updatedRows': 2,
  'updatedColumns': 9,
  'updatedCells': 18}}